<a href="https://colab.research.google.com/github/dananjarukshan/ML_SLIIT_Course/blob/main/ML_NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
import pandas as pd

In [59]:
file_path = '/content/drive/My Drive/Colab Notebooks/SLIIT_ML_course_CODES/Final_Project_2_NLP/Tweets.csv'
df = pd.read_csv(file_path) # read the path and copy data to df variiable
df.head()  # read first five rows

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [60]:
print(df.shape)  # no of rows and columns in the dataset
rows = df.shape[0]  # no of rows in the dataset
print("no of rows: {}".format(rows))
columns = df.shape[1]  # no of columns in the dataset
print(columns)

(14640, 15)
no of rows: 14640
15


In [61]:
df = df[["airline_sentiment", "text"]]
print(df.head())

  airline_sentiment                                               text
0           neutral                @VirginAmerica What @dhepburn said.
1          positive  @VirginAmerica plus you've added commercials t...
2           neutral  @VirginAmerica I didn't today... Must mean I n...
3          negative  @VirginAmerica it's really aggressive to blast...
4          negative  @VirginAmerica and it's a really big bad thing...


In [62]:
import nltk
import string
import re
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")
from nltk.corpus import stopwords
nltk.download("punkt")
nltk.download('punkt_tab') # Download the punkt_tab resource
ps = PorterStemmer()

def clean_text(text):
  text = text.lower()
  text = re.sub(r'http.?//[^\s]+[\s]?','',text)
  text = nltk.word_tokenize(text)
  y =[]
  for i in text:
    if i not in stopwords.words('english'):
      y.append(i)
  text = y[:]
  y.clear()
  for i in text:
    y.append(ps.stem(i))
  return" ".join(y)

# Apply cleaning to text column
df['text_cleaned'] = df['text'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


**feature extraction**

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=3000)
X = tfidf.fit_transform(df['text_cleaned']).toarray()
y = df['airline_sentiment'].values

**Model Training and Evaluation**

In [64]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

**Train and evaluate Multinomial Naive Bayes:**

In [65]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)
print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))

Naive Bayes Accuracy: 0.7226775956284153


**Train and evaluate Random Forest:**

In [66]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

Random Forest Accuracy: 0.7489754098360656


--------------------------------------------------------------------------------------------


**Preprocess the New Data for testing**

In [67]:
def preprocess_new_text(text):
    # Apply the same cleaning function used during training
    cleaned_text = clean_text(text)
    return cleaned_text

# Example new tweets to test
new_tweets = [
    "@VirginAmerica your flight attendants are so friendly! Great service!",
    "@VirginAmerica worst flight experience ever. Delayed for 5 hours.",
    "@VirginAmerica my flight was okay, nothing special"
]

# Clean the new tweets
cleaned_new_tweets = [preprocess_new_text(tweet) for tweet in new_tweets]

**Transform the Text to Features**
* Use the same TF-IDF vectorizer that was fit on the training data:

In [68]:
# Transform the new text using the already-fitted TF-IDF vectorizer
X_new = tfidf.transform(cleaned_new_tweets).toarray()

** Make Predictions**
* Use your trained model to predict sentiments:

In [69]:
# Using the Random Forest model (could also use Naive Bayes)
predictions = rf_model.predict(X_new)
probabilities = rf_model.predict_proba(X_new)  # Get confidence scores

# Display results
for tweet, pred, prob in zip(new_tweets, predictions, probabilities):
    print(f"Tweet: {tweet}")
    print(f"Predicted sentiment: {pred}")
    print(f"Confidence: {max(prob):.2f}")
    print("-" * 50)

Tweet: @VirginAmerica your flight attendants are so friendly! Great service!
Predicted sentiment: positive
Confidence: 0.68
--------------------------------------------------
Tweet: @VirginAmerica worst flight experience ever. Delayed for 5 hours.
Predicted sentiment: negative
Confidence: 0.80
--------------------------------------------------
Tweet: @VirginAmerica my flight was okay, nothing special
Predicted sentiment: negative
Confidence: 0.46
--------------------------------------------------


**Interpret Results**

The output will show:

* The original tweet

* Predicted sentiment (positive/negative/neutral)

* Confidence score (probability) of the prediction

In [70]:
def predict_sentiment(tweet, model=rf_model, vectorizer=tfidf):
    """
    Predict sentiment of a new tweet

    Parameters:
    tweet (str): The new tweet to analyze
    model: The trained classifier (default: random forest)
    vectorizer: The fitted TF-IDF vectorizer

    Returns:
    dict: Prediction results with confidence
    """
    # Preprocess
    cleaned = preprocess_new_text(tweet)

    # Transform
    features = vectorizer.transform([cleaned]).toarray()

    # Predict
    pred = model.predict(features)[0]
    proba = model.predict_proba(features)[0]
    confidence = max(proba)

    return {
        'tweet': tweet,
        'sentiment': pred,
        'confidence': float(confidence),
        'probabilities': {
            'negative': float(proba[0]),
            'neutral': float(proba[1]),
            'positive': float(proba[2])
        }
    }

**Example usage**

In [71]:
# Test the function
test_tweet = "@VirginAmerica the food was terrible but the crew was nice"
result = predict_sentiment(test_tweet)

print(f"Tweet: {result['tweet']}")
print(f"Predicted sentiment: {result['sentiment']}")
print(f"Confidence: {result['confidence']:.2f}")
print("Detailed probabilities:")
for sent, prob in result['probabilities'].items():
    print(f"{sent}: {prob:.2f}")

Tweet: @VirginAmerica the food was terrible but the crew was nice
Predicted sentiment: neutral
Confidence: 0.54
Detailed probabilities:
negative: 0.23
neutral: 0.54
positive: 0.23


In [76]:
# Interactive testing loop
print("Airline Tweet Sentiment Analysis")
print("Enter a tweet to analyze (or 'quit' to exit):")

while True:
    user_input = input("\nYour tweet: ")

    if user_input.lower() == 'quit':
        break

    if not user_input.strip():
        print("Please enter some text!")
        continue

    result_rf = predict_sentiment(user_input, model=rf_model)  # Random Forest prediction
    result_nb = predict_sentiment(user_input, model = nb_model)  # Naive Bayes prediction


    print("\n=== Analysis Results ===")
    print(f"Original text: {result_rf['tweet']}\n")

    # Naive Bayes results
    print("Naive Bayes Model:")
    print(f"Predicted sentiment: {result_nb['sentiment']}")
    print(f"Confidence: {result_nb['confidence']:.2f}")
    print("Probabilities:")
    for sent, prob in result_nb['probabilities'].items():
        print(f"  {sent}: {prob:.4f}")

    # Random Forest results
    print("\nRandom Forest Model:")
    print(f"Predicted sentiment: {result_rf['sentiment']}")
    print(f"Confidence: {result['confidence']:.2f}")
    print("Probabilities:")
    for sent, prob in result_rf['probabilities'].items():
        print(f"  {sent}: {prob:.4f}")

    print("\n" + "="*40 + "\n")

print("Goodbye!")

Airline Tweet Sentiment Analysis
Enter a tweet to analyze (or 'quit' to exit):

Your tweet: quit
Goodbye!
